# 🧪 Preprocessing Validation Notebook — 3번 역할 전용

### 3번 역할의 핵심 업무
1. `baseline_stats` 수집 → 2번 파이프라인 결과가 **설계와 일치하는지 비교**할 기준값 만들기  
2. 설계서에 정의된 **Feature Engineering(FE) 후보 4개** 검증  
   - `engagement_score`  
   - `listening_time_bin`  
   - `skip_rate_cap`  
   - `ads_pressure`  
3. (선택) 타깃 분포 확인 → 데이터 불균형 정도만 파악  
4. 2번이 만든 `preprocess_pipeline()` 이 **정상 동작하는지 QA 코드로 검증**


## 1. 원본 데이터 로드

**전체 EDA를 다시 하지 않고**,  
`baseline_stats`와 FE 검증에 필요한 최소한의 정보만 확인합니다.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)

#  필요에 따라 경로 수정
DATA_PATH = '../data/raw_data.csv'

df = pd.read_csv(DATA_PATH)
df.head()

,user_id,gender,age,country,subscription_type,listening_time,songs_played_per_day,skip_rate,device_type,ads_listened_per_week,offline_listening,is_churned
0,1,Female,54,CA,Free,26.0,23.0,0.20,Desktop,31,0,1
1,2,Other,33,DE,Family,141.0,62.0,0.34,Web,0,1,0
2,3,Male,38,AU,Premium,199.0,38.0,0.04,Mobile,0,1,1
3,4,Female,22,CA,Student,36.0,2.0,0.31,Mobile,0,1,0
4,5,Other,29,US,Family,250.0,57.0,0.36,Mobile,0,1,1


## 2. Baseline Stats (전처리 검증용 기준값)

2번 파이프라인이 올바르게 처리되었는지 확인하기 위해,  
원본 데이터 기준으로 **핵심 통계값들을 한 번만 계산**해 둡니다.

- `median` / `max` / `q99` (상위 1% 기준)  
  - `listening_time`  
  - `songs_played_per_day`  
  - `skip_rate`  
- `country_top5` (이용자 수 상위 5개 국가)
- `subscription_type_unique`
- `device_type_unique`


In [2]:
baseline_stats = {}

# 2-1. 수치형 기준값
num_cols = ['listening_time', 'songs_played_per_day', 'skip_rate']
for col in num_cols:
    if col in df.columns:
        baseline_stats[f'{col}_median'] = df[col].median()
        baseline_stats[f'{col}_max'] = df[col].max()
        baseline_stats[f'{col}_q99'] = df[col].quantile(0.99)

# 2-2. 범주형 기준값
if 'country' in df.columns:
    baseline_stats['country_top5'] = df['country'].value_counts().head(5).index.tolist()

if 'subscription_type' in df.columns:
    baseline_stats['subscription_type_unique'] = df['subscription_type'].unique().tolist()

if 'device_type' in df.columns:
    baseline_stats['device_type_unique'] = df['device_type'].unique().tolist()

baseline_stats

{'listening_time_median': np.float64(154.0),
 'listening_time_max': np.float64(299.0),
 'listening_time_q99': np.float64(297.0),
 'songs_played_per_day_median': np.float64(50.0),
 'songs_played_per_day_max': np.float64(99.0),
 'songs_played_per_day_q99': np.float64(98.0),
 'skip_rate_median': np.float64(0.305),
 'skip_rate_max': np.float64(4.980194893630863),
 'skip_rate_q99': np.float64(3.1209042144663313),
 'country_top5': ['AU', 'US', 'DE', 'IN', 'PK'],
 'subscription_type_unique': ['Free', 'Family', 'Premium', 'Student'],
 'device_type_unique': ['Desktop', 'Web', 'Mobile']}

### 2-3. (선택) baseline_stats를 JSON 파일로 저장

나중에 2번 파이프라인 결과와 비교할 때,  
같은 기준을 재사용하기 쉽도록 파일로 저장해 둡니다.


In [3]:
import json

with open('../data/baseline_stats.json', 'w', encoding='utf-8') as f:
    json.dump(baseline_stats, f, ensure_ascii=False, indent=2)

print('✅ baseline_stats.json 저장 완료')

✅ baseline_stats.json 저장 완료


## 3. Target 분포 확인 (선택)
**필수 업무는 아니지만**,  
데이터 불균형 정도를 알고 있으면 **모델 성능 해석**에 도움이 됩니다.


In [4]:
if 'is_churned' in df.columns:
    print('is_churned 분포 (비율):')
    print(df['is_churned'].value_counts(normalize=True).rename('ratio'))
else:
    print("⚠️ 'is_churned' 컬럼이 없습니다.")

is_churned 분포 (비율):
is_churned
0    0.741125
1    0.258875
Name: ratio, dtype: float64


## 4. Feature Engineering 후보 4개 실험

1번(설계자)이 정의한 FE 후보들 중에서,  
아래 4개를 실제로 만들어 보고 값이 정상적으로 계산되는지만 간단히 확인합니다.

- `engagement_score` : 듣는 시간 × 곡 수(활동 강도)
- `listening_time_bin` : 사용 시간을 low / mid / high로 구간화한 변수
- `skip_rate_cap` : 이상치를 줄이기 위해 cap(0~1.5)을 적용한 스킵률
- `ads_pressure` : 듣는 시간 대비 광고 노출 강도 (`ads_listened_per_week / listening_time`)

In [5]:
df_fe = df.copy()

# 4-1. engagement_score (활동 강도)
if {'listening_time', 'songs_played_per_day'}.issubset(df_fe.columns):
    # 설계서 정의: listening_time × songs_played_per_day
    df_fe['engagement_score'] = df_fe['listening_time'] * df_fe['songs_played_per_day']
else:
    print('⚠️ engagement_score를 만들기 위한 컬럼이 부족합니다.')

# 4-2. listening_time_bin (low / mid / high)
if 'listening_time' in df_fe.columns:
    try:
        # 전체 분포를 3등분해서 low/mid/high 구간으로 나눔
        df_fe['listening_time_bin'] = pd.qcut(df_fe['listening_time'], 3, labels=['low', 'mid', 'high'])
    except Exception as e:
        # qcut이 실패하면 단순 구간 기준으로 나누기
        bins = [0, 60, 180, df_fe['listening_time'].max()]
        labels = ['low', 'mid', 'high']
        df_fe['listening_time_bin'] = pd.cut(df_fe['listening_time'], bins=bins, labels=labels, include_lowest=True)
else:
    print('⚠️ listening_time 컬럼이 없습니다.')

# 4-3. skip_rate_cap (0 ~ 1.5 cap)
if 'skip_rate' in df_fe.columns:
    df_fe['skip_rate_cap'] = df_fe['skip_rate'].clip(lower=0, upper=1.5)
else:
    print('⚠️ skip_rate 컬럼이 없습니다.')

# 4-4. ads_pressure = ads_listened_per_week / listening_time
if {'ads_listened_per_week', 'listening_time'}.issubset(df_fe.columns):
    # 설계서 정의를 따르되, listening_time이 0인 경우는 NaN 처리
    lt_nonzero = df_fe['listening_time'].replace(0, np.nan)
    df_fe['ads_pressure'] = df_fe['ads_listened_per_week'] / lt_nonzero
else:
    print('⚠️ ads_pressure를 만들기 위한 컬럼이 부족합니다.')

df_fe[['engagement_score', 'listening_time_bin', 'skip_rate_cap', 'ads_pressure']].head()

,engagement_score,listening_time_bin,skip_rate_cap,ads_pressure
0,598.0,low,0.20,1.192308
1,8742.0,mid,0.34,0.000000
2,7562.0,mid,0.04,0.000000
3,72.0,low,0.31,0.000000
4,14250.0,high,0.36,0.000000


## 5. FE 변수와 이탈률의 **간단한 관계만** 확인

3번 역할은 **복잡한 통계 분석까지는 하지 않고**,  
각 파생변수별로 이탈률 평균만 비교해서 **대략적으로 유의미한지 감만 보는 수준**만 수행합니다.

여기서는 설계서 기준 FE 4개에 대해서만 확인합니다.
- `engagement_score`
- `listening_time_bin`
- `skip_rate_cap`
- `ads_pressure`


In [6]:
if 'is_churned' in df_fe.columns:
    for col in ['engagement_score', 'listening_time_bin', 'skip_rate_cap', 'ads_pressure']:
        if col in df_fe.columns:
            print(f'\n=== {col} vs is_churned ===')
            
            # 범주형(또는 구간형) 변수: listening_time_bin
            if col == 'listening_time_bin' or df_fe[col].dtype in ['int64', 'int32', 'bool', 'category', 'object']:
                print(df_fe.groupby(col)['is_churned'].mean().rename('churn_rate'))
            else:
                # 연속형은 단순 4분위 구간별 평균만 확인
                try:
                    bins = pd.qcut(df_fe[col], 4, duplicates='drop')
                    print(df_fe.groupby(bins)['is_churned'].mean().rename('churn_rate'))
                except Exception as e:
                    print('  (분위수 분할 불가, 분포가 한쪽에 치우쳤을 수 있음)')
else:
    print("⚠️ 'is_churned' 컬럼이 없어 FE-타깃 관계를 계산할 수 없습니다.")


=== engagement_score vs is_churned ===
engagement_score
(9.999, 2328.0]       0.248540
(2328.0, 5891.0]      0.271520
(5891.0, 11722.5]     0.263830
(11722.5, 29601.0]    0.254516
Name: churn_rate, dtype: float64

=== listening_time_bin vs is_churned ===
listening_time_bin
low     0.260718
mid     0.265715
high    0.252133
Name: churn_rate, dtype: float64

=== skip_rate_cap vs is_churned ===
skip_rate_cap
(-0.001, 0.15]    0.256284
(0.15, 0.305]     0.245053
(0.305, 0.46]     0.260048
(0.46, 1.5]       0.274419
Name: churn_rate, dtype: float64

=== ads_pressure vs is_churned ===
ads_pressure
(-0.001, 0.0239]    0.263230
(0.0239, 3.9]       0.248454
Name: churn_rate, dtype: float64


C:\Users\jiyon\AppData\Local\Temp\ipykernel_15596\1144278135.py:13: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  print(df_fe.groupby(bins)['is_churned'].mean().rename('churn_rate'))
C:\Users\jiyon\AppData\Local\Temp\ipykernel_15596\1144278135.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  print(df_fe.groupby(col)['is_churned'].mean().rename('churn_rate'))
C:\Users\jiyon\AppData\Local\Temp\ipykernel_15596\1144278135.py:13: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to

## 6. FE 개선안 메모

위 결과를 본 뒤, **3번 역할이 간단히 코멘트만 남기는 공간**입니다.

- `engagement_score` :  
  - 활동 강도가 아주 낮은 구간과 높은 구간의 이탈률이 큰 차이는 없었음
  - 다만 사용자 “활동성”을 설명하기에는 직관적인 지표라, 다른 변수와 함께 쓸 여지는 있음
  - → **효과는 약하지만, 해석용/보조용으로는 검토 가능**

- `listening_time_bin` :  
  - low / mid / high 구간별 이탈률 차이가 크지 않았음
  - 단순 구간화라서 모델에 큰 도움이 되기보다는, 리포트/설명용으로 쓰기 좋음
  - → **모델 핵심 변수라기보다는 분석용(시각화·설명용)으로 두는 것을 추천**

- `skip_rate_cap` :  
  - cap(0~1.5) 적용으로 극단적인 이상치가 줄어들어 분포가 안정됨
  - 자체적으로 churn 예측력을 크게 올리지는 않지만, 다른 FE(`high_skip`, `skip_intensity` 등)를 만들기 위한 
    “정리된 기반 변수”로 의미가 큼
  - → **전처리 단계에서 cap 적용은 유지 추천 (단, 파생 변수는 후속 실험에서 선택)**

- `ads_pressure` :  
  - 광고 노출 강도(ads_listened_per_week / listening_time)가 높다고 해서 이탈률이 크게 달라지지는 않을 수 있음
  - 다만 “광고를 많이 듣는 헤비 유저” 군집을 구분하거나, 다른 변수와 상호작용을 볼 때는 의미가 있을 수 있음
  - → **단독 영향력은 약하지만, 분석/군집용 보조 변수로는 검토 가능**


## 6-1. 추가 FE 후보 제안 정리

역할 3(전처리 검증 + Feature 튜닝) 관점에서, 기존 FE 4개 외에 아래 파생 변수들을 추가 후보로 제안한다.

- `songs_per_minute` = songs_played_per_day / listening_time  
  - 같은 사용 시간 대비 더 자주 곡을 재생하는 **활동성/집중도 지표**
- `skip_intensity` = skip_rate_cap × songs_played_per_day  
  - 단순 비율(skip_rate)보다 실제 **스킵 행동량**을 반영하는 지표
- `high_skip_flag` (예: skip_rate_cap > 0.5인 경우 1)  
  - 스킵이 지나치게 많은 **위험 고객 구간**을 표시하는 이진 변수
- `high_ads_pressure_flag` (예: ads_pressure 상위 10% 이상이면 1)  
  - 광고 노출 강도가 높은 고객을 따로 태깅하여 **광고 피로도에 따른 이탈 패턴**을 보기 위함
- `offline_heavy_user` (offline_listening == 1 이면서 listening_time > median)  
  - 오프라인 기능까지 적극적으로 사용하는 **헤비 유저 집단**을 구분
- `age_group` (예: young / adult / senior)  
  - 연령대별 소비 패턴 차이를 반영하는 구간형 변수
- `subscription_type_level` (예: Free=0, Student=1, Premium=2, Family=3)  
  - 요금제의 **혜택/가격 레벨**을 순서형 수치로 표현하여, 설계서에서 관찰된 이탈률 순서를 모델에 전달


In [9]:
# 6-1. 추가 FE 후보 구현 

# 기존 4-번 셀에서 생성한 df_fe를 기반으로, 추가 파생 변수들을 계산한다.
df_fe_extra = df_fe.copy()

# 1) songs_per_minute = songs_played_per_day / listening_time
if {'songs_played_per_day', 'listening_time'}.issubset(df_fe_extra.columns):
    lt_nonzero = df_fe_extra['listening_time'].replace(0, np.nan)
    df_fe_extra['songs_per_minute'] = df_fe_extra['songs_played_per_day'] / lt_nonzero

# 2) skip_intensity = skip_rate_cap × songs_played_per_day
if {'skip_rate_cap', 'songs_played_per_day'}.issubset(df_fe_extra.columns):
    df_fe_extra['skip_intensity'] = df_fe_extra['skip_rate_cap'] * df_fe_extra['songs_played_per_day']

# 3) high_skip_flag (예: skip_rate_cap > 0.5)
if 'skip_rate_cap' in df_fe_extra.columns:
    df_fe_extra['high_skip_flag'] = (df_fe_extra['skip_rate_cap'] > 0.5).astype(int)

# 4) high_ads_pressure_flag (ads_pressure 상위 10%)
if 'ads_pressure' in df_fe_extra.columns:
    threshold_ads = df_fe_extra['ads_pressure'].quantile(0.9)
    df_fe_extra['high_ads_pressure_flag'] = (df_fe_extra['ads_pressure'] > threshold_ads).astype(int)

# 5) offline_heavy_user (offline_listening == 1 이면서 listening_time > median)
if {'offline_listening', 'listening_time'}.issubset(df_fe_extra.columns):
    median_lt = df_fe_extra['listening_time'].median()
    df_fe_extra['offline_heavy_user'] = (
        (df_fe_extra['offline_listening'] == 1) &
        (df_fe_extra['listening_time'] > median_lt)
    ).astype(int)

# 6) age_group (young / adult / senior)
if 'age' in df_fe_extra.columns:
    df_fe_extra['age_group'] = pd.cut(
        df_fe_extra['age'],
        bins=[0, 25, 45, df_fe_extra['age'].max()],
        labels=['young', 'adult', 'senior'],
        include_lowest=True
    )

# 7) subscription_type_level (Free=0, Student=1, Premium=2, Family=3)
if 'subscription_type' in df_fe_extra.columns:
    sub_map = {'Free': 0, 'Student': 1, 'Premium': 2, 'Family': 3}
    df_fe_extra['subscription_type_level'] = df_fe_extra['subscription_type'].map(sub_map)

# 계산된 추가 FE들의 상위 5개만 확인
cols_new_fe = [
    'songs_per_minute', 'skip_intensity', 'high_skip_flag',
    'high_ads_pressure_flag', 'offline_heavy_user',
    'age_group', 'subscription_type_level'
]

existing_cols = [c for c in cols_new_fe if c in df_fe_extra.columns]
df_fe_extra[existing_cols].head()


,songs_per_minute,skip_intensity,high_skip_flag,high_ads_pressure_flag,offline_heavy_user,age_group,subscription_type_level
0,0.884615,4.60,0,1,0,senior,0
1,0.439716,21.08,0,0,0,adult,3
2,0.190955,1.52,0,0,1,adult,2
3,0.055556,0.62,0,0,0,young,1
4,0.228000,20.52,0,0,1,adult,3


### 설계서에는 없지만, 모델/해석/QA 관점에서 있으면 좋은 형태로 몇 개를 더 만듦

- high_skip_flag
이유: skip_rate_cap는 연속형이라 해석이 애매해서,
“스킵이 진짜 많은 위험 고객”을 0/1로 뚝 잘라서 태깅하면 리포트/시각화에서 한눈에 보기 좋고
로지스틱/트리에서도 “고위험 구간” 더미변수로 쓰기 좋음.

- high_ads_pressure_flag
이유: ads_pressure도 연속형이라, 이걸 상위 10% 같은 구간 기준으로 이진 플래그 만들면
“광고 과다 노출 그룹 vs 나머지”로 쉽게 비교 가능
마케팅/정책 쪽에서 액션을 설계하기 쉬워짐.

- offline_heavy_user
이유: offline_listening(0/1)과 listening_time(연속형)을 그냥 따로 쓰면
“오프라인까지 적극적으로 쓰는 진짜 헤비유저”를 집어내기 어렵고, 두 변수를 묶어서
기능을 깊게 쓰는 충성 고객인지 혹은 요금/광고 때문에 나갈 수 있는 위험 헤비유저인지 를 따로 볼 수 있어서 군집/해석용으로 의미가 큼.
"""

## 6-2. 추가 FE와 이탈률(churn_rate) 간단 검증

위 6-1에서 정의한 추가 파생 변수들이 실제로 이탈률과 어떤 방향으로 관련이 있는지,
기본적인 **구간별 churn_rate(평균 이탈률)**만 확인한다.

- 대상 변수:
  - `songs_per_minute`
  - `skip_intensity`
  - `high_skip_flag`
  - `high_ads_pressure_flag`
  - `offline_heavy_user`
  - `age_group`
  - `subscription_type_level`

※ 이 셀은 **6-1 추가 FE 구현 셀(df_fe_extra 생성)을 먼저 실행한 후**에 돌려야 한다.


In [10]:
# 6-2. 추가 FE vs churn_rate 간단 검증

if 'is_churned' in df_fe_extra.columns:
    cols_check = [
        'songs_per_minute', 'skip_intensity', 'high_skip_flag',
        'high_ads_pressure_flag', 'offline_heavy_user',
        'age_group', 'subscription_type_level'
    ]

    for col in cols_check:
        if col not in df_fe_extra.columns:
            continue

        print(f"\n=== {col} vs is_churned ===")

        series = df_fe_extra[col]

        # 범주형/이진/정수형 변수는 그룹별 평균 이탈률을 바로 계산
        if series.dtype in ['int64', 'int32', 'bool', 'category', 'object'] or col in [
            'high_skip_flag', 'high_ads_pressure_flag', 'offline_heavy_user', 'age_group'
        ]:
            print(df_fe_extra.groupby(col)['is_churned'].mean().rename('churn_rate'))
        else:
            # 연속형 변수는 분위수 기반 구간(bin)으로 나눈 뒤 구간별 평균 이탈률 확인
            try:
                bins = pd.qcut(series, 4, duplicates='drop')
                print(df_fe_extra.groupby(bins)['is_churned'].mean().rename('churn_rate'))
            except Exception as e:
                print('  (분위수 분할 불가, 분포가 한쪽에 치우쳤을 수 있음)')
else:
    print("⚠️ 'is_churned' 컬럼이 없어 추가 FE-타깃 관계를 계산할 수 없습니다.")



=== songs_per_minute vs is_churned ===
songs_per_minute
(0.00238, 0.165]    0.249734
(0.165, 0.324]      0.258236
(0.324, 0.62]       0.270069
(0.62, 9.9]         0.260361
Name: churn_rate, dtype: float64

=== skip_intensity vs is_churned ===
skip_intensity
(-0.001, 4.258]    0.248969
(4.258, 11.57]     0.258763
(11.57, 23.76]     0.248454
(23.76, 148.5]     0.280412
Name: churn_rate, dtype: float64

=== high_skip_flag vs is_churned ===
high_skip_flag
0    0.255694
1    0.273692
Name: churn_rate, dtype: float64

=== high_ads_pressure_flag vs is_churned ===
high_ads_pressure_flag
0    0.258167
1    0.265464
Name: churn_rate, dtype: float64

=== offline_heavy_user vs is_churned ===
offline_heavy_user
0    0.258817
1    0.258978
Name: churn_rate, dtype: float64

=== age_group vs is_churned ===
age_group
young     0.247525
adult     0.264345
senior    0.259425
Name: churn_rate, dtype: float64

=== subscription_type_level vs is_churned ===
subscription_type_level
0    0.249257
1    0.26186

C:\Users\jiyon\AppData\Local\Temp\ipykernel_15596\731996954.py:27: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  print(df_fe_extra.groupby(bins)['is_churned'].mean().rename('churn_rate'))
C:\Users\jiyon\AppData\Local\Temp\ipykernel_15596\731996954.py:27: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  print(df_fe_extra.groupby(bins)['is_churned'].mean().rename('churn_rate'))
C:\Users\jiyon\AppData\Local\Temp\ipykernel_15596\731996954.py:22: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or obser

## 7. 간단 모델로 FE 성능 영향도 확인

2번 역할의 최종 전처리 파이프라인을 받기 전, 3번 역할 관점에서
**임시 전처리 + 간단 모델(Logistic Regression)** 만으로
"FE 포함 여부에 따라 성능이 얼마나 달라지는지"를 대략적으로 확인한다.

- 비교 대상
  - **기본 모델 (Base)**: 원본 주요 변수만 사용
  - **FE 확장 모델 (FE)**: Base + 설계/제안된 FE 변수들 추가
- 사용 데이터
  - `df_fe_extra` (6-1 셀에서 생성된, 원본 + FE가 합쳐진 DataFrame)
- 지표
  - Accuracy, ROC-AUC, F1-score (train/test split 동일 조건)

※ 이 섹션의 코드는 **최종 파이프라인/모델을 대체하지 않고**,
전처리/FE 아이디어의 "대략적인 방향성"만 체크하기 위한 참고용 실험이다.


In [12]:
# 7. 간단 모델로 FE 성능 영향도 확인

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score
from sklearn.impute import SimpleImputer

# df_fe_extra가 6-1 셀에서 생성되었는지 확인
try:
    _ = df_fe_extra
except NameError:
    raise RuntimeError("df_fe_extra가 없습니다. 먼저 6-1 셀(추가 FE 구현)을 실행해 주세요.")

# 타깃 변수
if 'is_churned' not in df_fe_extra.columns:
    raise RuntimeError("'is_churned' 컬럼이 없습니다. 원본 데이터 로드/전처리를 다시 확인해 주세요.")

y = df_fe_extra['is_churned']

# 1) 기본(Base) Feature 세트 정의
base_cols_candidate = [
    'age', 'listening_time', 'songs_played_per_day', 'skip_rate',
    'ads_listened_per_week', 'gender', 'country',
    'subscription_type', 'device_type', 'offline_listening'
]
base_cols = [c for c in base_cols_candidate if c in df_fe_extra.columns]

# 2) FE 확장 Feature 세트 정의 (기본 + FE)
fe_cols_candidate = [
    'engagement_score', 'listening_time_bin', 'skip_rate_cap', 'ads_pressure',
    'songs_per_minute', 'skip_intensity', 'high_skip_flag',
    'high_ads_pressure_flag', 'offline_heavy_user', 'age_group',
    'subscription_type_level'
]
fe_cols = [c for c in fe_cols_candidate if c in df_fe_extra.columns]

print("[사용 Feature 세트]")
print("Base cols:", base_cols)
print("FE cols  :", fe_cols)


def make_pipeline(X: "pd.DataFrame"):
    """수치/범주형 컬럼을 자동으로 나눠 간단 전처리 + 로지스틱 회귀 파이프라인 구성"""
    numeric_features = [
        col for col in X.columns
        if X[col].dtype in ['int64', 'float64']
    ]
    categorical_features = [
        col for col in X.columns
        if X[col].dtype not in ['int64', 'float64']
    ]

    numeric_transformer = Pipeline(
        steps=[
            ('imputer', SimpleImputer(strategy='median'))
        ]
    )
    categorical_transformer = Pipeline(
        steps=[
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('onehot', OneHotEncoder(handle_unknown='ignore'))
        ]
    )

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features)
        ]
    )

    clf = LogisticRegression(max_iter=1000)

    pipe = Pipeline(
        steps=[('preprocess', preprocessor), ('model', clf)]
    )
    return pipe


def eval_with_features(feature_cols, label: str):
    X = df_fe_extra[feature_cols].copy()
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )

    pipe = make_pipeline(X_train)
    pipe.fit(X_train, y_train)

    y_pred = pipe.predict(X_test)
    y_proba = pipe.predict_proba(X_test)[:, 1]

    acc = accuracy_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_proba)
    f1 = f1_score(y_test, y_pred)

    print(f"\n[{label}]")
    print(f"Accuracy: {acc:.4f}")
    print(f"ROC-AUC : {auc:.4f}")
    print(f"F1-score: {f1:.4f}")


# Base vs FE 모델 비교 실행
if base_cols:
    eval_with_features(base_cols, 'Base (원본 주요 변수만)')
else:
    print("⚠️ Base feature 컬럼이 없어 Base 모델을 학습할 수 없습니다.")

if base_cols or fe_cols:
    eval_with_features(base_cols + fe_cols, 'Base + FE (추가 파생변수 포함)')
else:
    print("⚠️ 사용할 수 있는 Feature 컬럼이 없습니다.")


[사용 Feature 세트]
Base cols: ['age', 'listening_time', 'songs_played_per_day', 'skip_rate', 'ads_listened_per_week', 'gender', 'country', 'subscription_type', 'device_type', 'offline_listening']
FE cols  : ['engagement_score', 'listening_time_bin', 'skip_rate_cap', 'ads_pressure', 'songs_per_minute', 'skip_intensity', 'high_skip_flag', 'high_ads_pressure_flag', 'offline_heavy_user', 'age_group', 'subscription_type_level']

[Base (원본 주요 변수만)]
Accuracy: 0.7412
ROC-AUC : 0.4946
F1-score: 0.0000

[Base + FE (추가 파생변수 포함)]
Accuracy: 0.7412
ROC-AUC : 0.4974
F1-score: 0.0000


c:\Users\jiyon\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 1000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=1000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


## 7-1. 간단 성능 영향도 분석 결과 정리 (보고용 메모)

- **데이터 특성**
  - 타깃 분포: `is_churned=0` 약 74.1%, `is_churned=1` 약 25.9% → **불균형 데이터**

- **모델/실험 설정**
  - 모델: `LogisticRegression(max_iter=1000)`
  - 비교:
    - Base: 원본 주요 변수만 사용
    - Base+FE: Base + 설계/제안된 FE(engagement_score, skip_intensity 등) 추가

- **결과 요약**
  - Base: Accuracy ≈ 0.7412, ROC-AUC ≈ 0.4946, F1-score(양성 클래스 기준) ≈ 0.0
  - Base+FE: Accuracy ≈ 0.7412, ROC-AUC ≈ 0.4974, F1-score ≈ 0.0
  - 해석:
    - 불균형 데이터 + 기본 설정의 로지스틱 회귀에서는 **대부분을 0(미이탈)로 예측**하는 경향이 강함
    - 이 때문에 양성(1) 예측이 거의 없어 **F1-score(양성 기준)가 0에 수렴**
    - Base 대비 Base+FE의 AUC는 미세하게(0.49 → 0.497 수준)만 개선되어, **간단 로지스틱 기준에서는 FE 추가 효과가 크지 않음**

- **3번 역할 관점의 결론**
  - 현재 FE들은 **단순 로지스틱 단일 모델 기준으로는 강한 성능 향상을 보여주지 않음**
  - 다만, 불균형(class imbalance)과 모델 종류 영향이 크므로, 
    - 추후 4번 역할의 템플릿/트리 계열 모델(XGBoost/LightGBM 등)
    - 또는 `class_weight='balanced'` 등의 설정과 함께 **추가 실험 시 재검토 필요**
  - 따라서, 본 노트북에서는 **"FE 방향성 점검 + 간단한 성능 영향도 확인" 수준까지 수행**한 것으로 정리한다.


## 8. 2번 전처리 파이프라인 QA 템플릿

2번 역할이 `preprocess_pipeline()`을 완성한 이후,  
아래 코드를 주석 해제해서 **전처리 품질을 자동으로 점검**합니다.

검증 항목:
1. `X_train`, `X_test`, `y_train`, `y_test` shape 정상 여부  
2. 결측치가 모두 제거되었는지  
3. 타깃 분포가 원본과 크게 달라지지 않았는지  
4. 설계된 FE(`engagement_score`, `listening_time_bin`, `skip_rate_cap`, `ads_pressure`)가 실제 포함되어 있는지  
5. (선택) baseline_stats와 비교하여 이상치 처리가 되었는지


In [ ]:
# from preprocessing import preprocess_pipeline
# import json
#
# # baseline_stats 불러오기 (있다면)
# try:
#     with open('../data/baseline_stats.json', 'r', encoding='utf-8') as f:
#         baseline_loaded = json.load(f)
# except FileNotFoundError:
#     baseline_loaded = None
#     print('⚠️ baseline_stats.json 파일을 찾을 수 없습니다.')
#
# # 7-1. 파이프라인 실행
# X_train, X_test, y_train, y_test = preprocess_pipeline(df.copy())
#
# print('X_train shape:', X_train.shape)
# print('X_test shape :', X_test.shape)
# print('y_train shape:', y_train.shape)
# print('y_test shape :', y_test.shape)
#
# # 7-2. 결측치 검증
# print('\n[결측치 검증]')
# print('X_train 결측치 총합:', X_train.isnull().sum().sum())
# print('X_test  결측치 총합:', X_test.isnull().sum().sum())
#
# # 7-3. 타깃 분포 검증
# print('\n[y_train 분포]')
# print(y_train.value_counts(normalize=True).rename('ratio'))
#
# # 7-4. FE 생성 여부 확인
# print('\n[FE 컬럼 존재 여부]')
# required_fe = ['engagement_score', 'listening_time_bin', 'skip_rate_cap', 'ads_pressure']
# for col in required_fe:
#     print(f'  {col}:', col in X_train.columns)
#
# # 7-5. (선택) baseline과 이상치 비교
# if baseline_loaded is not None:
#     for col in ['skip_rate']:
#         if col in X_train.columns and f'{col}_q99' in baseline_loaded:
#             q99_orig = baseline_loaded[f'{col}_q99']
#             max_after = X_train[col].max()
#             print(f"\n[{col}] 원본 q99: {q99_orig:.4f}, 전처리 후 max: {max_after:.4f}")
#             if max_after > q99_orig * 1.5:
#                 print('  ⚠️ 이상치 처리가 부족할 수 있습니다.')

---
## ✔ 최종 정리 (3번 역할 관점)

이 노트북은 **3번 역할이 해야 할 것만 정확히 담은 적정 버전**입니다.

- ❌ 전체 EDA 반복하지 않음  
- ❌ 과도한 FE 생성·상관계수 분석 없음  
- ✅ baseline_stats 수집 → 나중에 전처리 결과와 비교 가능  
- ✅ 설계된 FE 4개만 검증 → 역할 범위 초과하지 않음  
- ✅ 추가 FE 제안 + churn_rate 패턴 확인 → 전처리/FE 방향성 사전 검증  
- ✅ (파이프라인 적용 전) 간단 로지스틱 모델로 Base vs Base+FE 성능 영향도 1차 점검  
- ✅ 파이프라인 QA 템플릿 제공 → 2번 코드가 설계대로 동작하는지 체크 가능  

> 참고: 7번 섹션의 성능 영향도 분석은 **전처리 파이프라인 적용 전 기준선(baseline)**으로,  
> 2번이 파이프라인을 완성한 이후에는 8번 QA 섹션에서 "파이프라인 적용 후" 관점의 추가 검증을 수행할 수 있습니다.
